In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.util import ngrams

In [10]:
df = pd.read_csv('data/tokenized_clean.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.replace(to_replace=r"\[|\]|\' ", value='', inplace=True, regex=True)
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this may allah fo...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive evacuation orders in california,1
4,7,NaN,NaN,just got sent this photo from ruby as smoke f...,1


In [45]:
freqs = {}
for i in df['text']:
    tokens = nltk.tokenize.word_tokenize(i, language="english")
    for n in range(1, 4):
        for j in ngrams(tokens, n):
            j = str(j)
            freqs.setdefault(j, 0)
            freqs[j]+=1
freqs

{"('our',)": 99,
 "('deeds',)": 2,
 "('are',)": 401,
 "('the',)": 3263,
 "('reason',)": 20,
 "('of',)": 1823,
 "('this',)": 477,
 "('may',)": 88,
 "('allah',)": 4,
 "('forgive',)": 2,
 "('us',)": 163,
 "('all',)": 257,
 "('our', 'deeds')": 1,
 "('deeds', 'are')": 1,
 "('are', 'the')": 15,
 "('the', 'reason')": 2,
 "('reason', 'of')": 2,
 "('of', 'this')": 9,
 "('this', 'may')": 1,
 "('may', 'allah')": 3,
 "('allah', 'forgive')": 1,
 "('forgive', 'us')": 1,
 "('us', 'all')": 5,
 "('our', 'deeds', 'are')": 1,
 "('deeds', 'are', 'the')": 1,
 "('are', 'the', 'reason')": 1,
 "('the', 'reason', 'of')": 1,
 "('reason', 'of', 'this')": 1,
 "('of', 'this', 'may')": 1,
 "('this', 'may', 'allah')": 1,
 "('may', 'allah', 'forgive')": 1,
 "('allah', 'forgive', 'us')": 1,
 "('forgive', 'us', 'all')": 1,
 "('forest',)": 63,
 "('fire',)": 247,
 "('near',)": 54,
 "('la',)": 25,
 "('ronge',)": 1,
 "('sask',)": 1,
 "('canada',)": 9,
 "('forest', 'fire')": 8,
 "('fire', 'near')": 3,
 "('near', 'la')": 1,


In [93]:
indexer = {}
index = 0
for i in df['text']:
    tokens = nltk.tokenize.word_tokenize(i, language="english")
    for n in range(1, 3):
        for j in ngrams(tokens, n):
            j = str(j)
            if j not in indexer.keys() and freqs[j] > 20:
                indexer[j] = index
                index+=1
indexer

{"('our',)": 0,
 "('are',)": 1,
 "('the',)": 2,
 "('of',)": 3,
 "('this',)": 4,
 "('may',)": 5,
 "('us',)": 6,
 "('all',)": 7,
 "('forest',)": 8,
 "('fire',)": 9,
 "('near',)": 10,
 "('la',)": 11,
 "('to',)": 12,
 "('in',)": 13,
 "('place',)": 14,
 "('being',)": 15,
 "('by',)": 16,
 "('no',)": 17,
 "('other',)": 18,
 "('evacuation',)": 19,
 "('or',)": 20,
 "('people',)": 21,
 "('california',)": 22,
 "('just',)": 23,
 "('got',)": 24,
 "('photo',)": 25,
 "('from',)": 26,
 "('as',)": 27,
 "('smoke',)": 28,
 "('into',)": 29,
 "('a',)": 30,
 "('school',)": 31,
 "('into', 'a')": 32,
 "('update',)": 33,
 "('both',)": 34,
 "('due',)": 35,
 "('county',)": 36,
 "('due', 'to')": 37,
 "('rain',)": 38,
 "('flash',)": 39,
 "('flooding',)": 40,
 "('im',)": 41,
 "('on',)": 42,
 "('top',)": 43,
 "('and',)": 44,
 "('i',)": 45,
 "('can',)": 46,
 "('see',)": 47,
 "('top', 'of')": 48,
 "('of', 'the')": 49,
 "('and', 'i')": 50,
 "('i', 'can')": 51,
 "('fire', 'in')": 52,
 "('in', 'the')": 53,
 "('theres',)"

In [48]:
len(indexer)*len(df)*4/10e9

0.0075551412

In [94]:
n_gram_onehot = np.zeros(shape=(len(df), len(indexer)), dtype=np.float32)
n_gram_onehot.shape

(7613, 1064)

In [50]:
import json
with open('indexes.sav', 'w') as f:
    f.write(json.dumps(indexer))

In [60]:
for i, e in enumerate(df['text']):
    tokens = nltk.tokenize.word_tokenize(e, language="english")
    for j in ngrams(tokens, n):
        if j in indexer.keys():
            n_gram_onehot[i][indexer[j]] = 1.0

In [71]:
labelsOneHot = np.zeros((n_gram_onehot.shape[0], 2))
for i, e in enumerate(df['target']):
    labelsOneHot[i][e]=1
labelsOneHot.shape

(7613, 2)

In [20]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [96]:
keras.utils.set_random_seed(1234)
ngram_NN = keras.Sequential()
ngram_NN.bias = 0.2
ngram_NN.add(keras.layers.Input(1064))

ngram_NN.add(keras.layers.Reshape((56, 19)))

ngram_NN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))#, padding="causal"))
ngram_NN.add(keras.layers.MaxPool1D(2))

ngram_NN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))#, padding="causal"))
ngram_NN.add(keras.layers.MaxPool1D(2))

ngram_NN.add(keras.layers.Flatten())

ngram_NN.add(keras.layers.Dense(512, activation='relu'))
ngram_NN.add(keras.layers.Dense(128,  activation='relu'))
ngram_NN.add(keras.layers.Dense(32,  activation='relu'))
ngram_NN.add(keras.layers.Dense(8,  activation='relu'))



ngram_NN.add(keras.layers.Dropout(.5))


ngram_NN.add(keras.layers.Dense(2, activation='softmax'))
ngram_NN.output_shape

(None, 2)

In [24]:
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

In [97]:
ngram_NN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [98]:
ngram_NN.fit(n_gram_onehot, labelsOneHot, epochs=25, callbacks=callbacks)

Epoch 1/25
238/238 [==============================] - 4s 10ms/step - loss: 0.6868 - accuracy: 0.5703
Epoch 2/25
238/238 [==============================] - 2s 9ms/step - loss: 0.6835 - accuracy: 0.5703
Epoch 3/25
238/238 [==============================] - 2s 8ms/step - loss: 0.6833 - accuracy: 0.5703
Epoch 4/25
238/238 [==============================] - 2s 8ms/step - loss: 0.6833 - accuracy: 0.5703
Epoch 5/25
238/238 [==============================] - 2s 7ms/step - loss: 0.6833 - accuracy: 0.5703
Epoch 6/25
238/238 [==============================] - 2s 7ms/step - loss: 0.6833 - accuracy: 0.5703
Epoch 7/25
238/238 [==============================] - 2s 7ms/step - loss: 0.6833 - accuracy: 0.5703
Epoch 8/25
238/238 [==============================] - 2s 9ms/step - loss: 0.6833 - accuracy: 0.5703
Epoch 9/25
238/238 [==============================] - 2s 9ms/step - loss: 0.6833 - accuracy: 0.5703
Epoch 10/25
238/238 [==============================] - 2s 7ms/step - loss: 0.6833 - accuracy: 0.570